In [1]:
import math
import copy
import numpy as np
import pandas as pd
from operator import itemgetter
from scipy import optimize

import matplotlib.pyplot as plt
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
%matplotlib inline

# Basic implementation of Nelder-Mead. 

In [148]:
# Original Paper: http://www.ime.unicamp.br/~sandra/MT853/handouts/Ref3(NelderMead1965).pdf
# Implemented from: http://www.webpages.uidaho.edu/~fuchang/res/ANMS.pdf
def Nelder_Mead(x0,func,
                step_size = 0.1,max_iterations=20,
                tol = 10e-6,no_improve_break=20):
    
    alpha = 1.0 # reflection
    beta  = 2.0 # contraction
    gamma = 0.5 # expansion
    
    N       = len(x0)
    x0      = np.array(x0)
    best    = func(x0)
    simplex = np.asarray(np.append(x0,best))

    no_improve    = 0

    # for adaptive NM we use
    #alpha = 1.0
    #beta  = 1.0 + 2.0/float(N)
    #gamma = 0.75 - 1.0/(2.0*float(N))
    #delta = 1.0 - 1.0/(float(N))
    
    # build initial simplex
    for i in range(N):
        x = np.array(x0, copy=True)
        if x[i] != 0:
            x[i] = (1 + step_size)*x[i] # from scipy
        else:
            x[i] = 0.00025
        simplex = np.concatenate((simplex,np.append(x,func(x))))
    simplex = simplex.reshape(int(len(simplex)/(N+1)),(N+1))
    simplex = simplex[simplex[:,N].argsort()] # ascending


    
    iterations = 0
    while True:
        # sort vertices by function value
        simplex = simplex[simplex[:,N].argsort()] # ascending
        min_value = simplex[0,-1]
        
        if iterations >= max_iterations and max_iterations >= max_iterations:
            return simplex[0,:]
        iterations += 1
        
        # break after no_improv_break iterations with no improvement
        print('...best so far:', min_value)

        if min_value < best - tol:
            no_improv = 0
            best = min_value
        else:
            no_improve += 1

        if no_improve >= no_improve_break:
            return simplex[0,:]
        
        centroid = np.array([np.sum(simplex[:-1,i]) for i in range(N)])
        
        # relection
        x_reflection = (1 + alpha)*centroid - alpha*simplex[-1,:-1]
        y_reflection = func(x_reflection)
        execute_shrink = 0
        
        # expansion
        if y_reflection < simplex[0,-1]:
            x_expansion = (1 + beta)*centroid - beta*simplex[-1,:-1]
            y_expansion = func(x_expansion)
            print("expansion")

            if y_expansion < y_reflection: # mistake in NL original paper at this step
                simplex[-1,:] = np.append(x_expansion,y_expansion)
                continue
            else:    
                simplex[-1,:] = np.append(x_reflection,y_reflection)
                continue
        
        else: # refelection is bigger than current min
            if  y_reflection < simplex[-2,-1]: # different than original paper
                print("reflection")
                simplex[-1,:] = np.append(x_reflection,y_reflection)
                continue
            else: # reflection is bigger than simplex[-2,-1]
                # contraction
                if y_reflection < simplex[-1,-1]:
                    print("outside contraction")
                    x_contraction_o = -gamma*simplex[-1,:-1] + (1 + gamma)*centroid
                    y_contraction_o = func(x_contraction_o)
                    
                    if y_contraction_o <= y_reflection:
                        simplex[-1,:] = np.append(x_contraction_o,y_contraction_o)
                        continue
                    else:
                        execute_shrink = 1
                else:
                    print("inside contraction")
                    x_contraction_i = gamma*simplex[-1,:-1] + (1 - gamma)*centroid
                    y_contraction_i = func(x_contraction_i)
                    
                    if y_contraction_i < simplex[-1,-1]:
                        simplex[-1,:] = np.append(x_contraction_i,y_contraction_i)
                        continue
                    else:
                        execute_shrink = 1
                    
                    if execute_shrink:
                        # shrink
                        low_vertex  = simplex[0,:-1]
                        new_simplex = np.zeros((N+1,N+1))
                        
                        for i,row in enumerate(simplex[:,:-1]):
                            x_new = low_vertex + (row - low_vertex)/2.0
                            y_new = func(x_new)
                            new_simplex[i,:] = np.asarray(np.append(x_new,y_new)) # maybe overwriting in loop....
                        simplex = new_simplex
                        simplex = simplex[simplex[:,N].argsort()]
                        continue


In [149]:
def func(x):
    return math.sin(x[0]) * math.cos(x[1]) * (1. / (abs(x[2]) + 1))
def fg(x):
    return 1/x[0] + x[1]*x[1] + x[1]*x[2]

In [150]:
x0 = [-1.2,4.0,3.0]
results = Nelder_Mead(x0,func)
results

...best so far: 0.0716115618056
expansion
...best so far: -0.00875917918085
reflection
...best so far: -0.00875917918085
reflection
...best so far: -0.00875917918085
reflection
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so far: -0.00875917918085
outside contraction
...best so fa

array([ -5.88000000e+00,   2.08000000e+01,   1.56000000e+01,
        -8.75917918e-03])

In [143]:
method = 'Nelder-Mead'
#start_time = time()
result = optimize.minimize(func, x0, method=method, tol=1e-6)
print("Scipy found a min of ",func(result.x),"at ",result.x)
#end_time = time()

Scipy found a min of  -0.999999999999 at  [ -1.57079640e+00  -2.08046050e-07   5.74031399e-13]


In [104]:
result

 final_simplex: (array([[ -1.57079651e+00,   1.06000404e-07,  -3.96509483e-13],
       [ -1.57079653e+00,   1.35072109e-07,   5.04471128e-13],
       [ -1.57079560e+00,  -3.26588866e-07,  -2.55802313e-13],
       [ -1.57079597e+00,   4.67627779e-07,  -5.98259979e-13]]), array([-1., -1., -1., -1.]))
           fun: -0.99999999999958167
       message: 'Optimization terminated successfully.'
          nfev: 276
           nit: 160
        status: 0
       success: True
             x: array([ -1.57079651e+00,   1.06000404e-07,  -3.96509483e-13])

[[ 0.          0.          0.          0.        ]
 [ 0.08054818  0.          0.          0.08046111]
 [ 0.          0.09752268  0.          0.        ]
 [ 0.          0.          0.05567404  0.        ]]


array([ 0.,  0.,  0.])

Scipy found a min of  -0.999999999992


In [751]:
centroid = np.array([np.sum(simplex[:-1,i]) for i in range(dimensions)])


In [783]:
result.x

array([ -1.57079402e+00,  -3.10687805e-06,   4.64004670e-13])

In [ ]:
    # sanity check to make sure we don't have a degenerate simplex
    # not sure this is necessary - check later
    bool_func_vals = any(simplex[:,dimensions])
    
    while not bool_func_vals:
        x = copy.copy(x0)
        index = np.random.randint(0,dimensions+1)
        x[index] = x[index] + step_size*np.random.uniform(-10, 10)
        simplex[-1,:] = np.append(x,func(x))
        
        if any(simplex[:,dimensions]):
            break

In [696]:
[np.sum(simplex[:-1,i]) for i in range(dimensions)]

[0.080548178084059327, 0.097522684890255423, 0.0]

In [31]:
# The Rosenbrock function
def Rosen(x):
    x = asarray(x)
    r = numpy.sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0,
                  axis=0)
    return r

In [130]:
# shrink
low_vertex  = simplex[0,:-1]
print(low_vertex)
new_simplex = np.zeros((N+1,N+1))
for i,row in enumerate(simplex[:,:-1]):
    print(row,low_vertex+(row - low_vertex)/2.0)
    x_new = low_vertex + (row - low_vertex)/2.0
    y_new = func(x_new)
    new_simplex[i,:] = np.asarray(np.append(x_new,y_new)) # maybe overwriting in loop....
new_simplex

[-1.32  1.    1.  ]
[-1.32  1.    1.  ] [-1.32  1.    1.  ]
[-1.2  1.   1. ] [-1.26  1.    1.  ]
[-1.2  1.   1.1] [-1.26  1.    1.05]
[-1.2  1.1  1. ] [-1.26  1.05  1.  ]


array([[-1.32      ,  1.        ,  1.        , -0.2616995 ],
       [-1.26      ,  1.        ,  1.        , -0.2572083 ],
       [-1.26      ,  1.        ,  1.05      , -0.25093493],
       [-1.26      ,  1.05      ,  1.        , -0.23686629]])

In [120]:
alpha = 1.0 # reflection
beta  = 2.0 # contraction
gamma = 0.5 # expansion

N       = len(x0)
x0      = np.array(x0)
best    = func(x0)
simplex = np.asarray(np.append(x0,best))

no_improve    = 0

# for adaptive NM we use
alpha = 1.0
beta  = 1.0 + 2.0/float(N)
gamma = 0.75 - 1.0/(2.0*float(N))
delta = 1.0 - 1.0/(float(N))

# build initial simplex
for i in range(N):
    x = np.array(x0, copy=True)
    if x[i] != 0:
        x[i] = (1 + 0.1)*x[i] # from scipy
    else:
        x[i] = 0.00025
    simplex = np.concatenate((simplex,np.append(x,func(x))))
simplex = simplex.reshape(int(len(simplex)/(N+1)),(N+1))
simplex = simplex[simplex[:,N].argsort()] # ascending

In [121]:
simplex

array([[-1.32      ,  1.        ,  1.        , -0.2616995 ],
       [-1.2       ,  1.        ,  1.        , -0.25179143],
       [-1.2       ,  1.        ,  1.1       , -0.23980137],
       [-1.2       ,  1.1       ,  1.        , -0.21138466]])